In [ ]:
import sys, os, subprocess
import numpy as np
import pandas as pd
import ipywidgets as w
import re
from IPython.display import display, clear_output, HTML
from openpyxl import load_workbook
import time
import warnings
import platform
import pyperclip

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 50)

warnings.filterwarnings('ignore')

get_ipython().run_cell_magic('javascript', '', 'IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}\n')

#------------------------------------------------------------------------------------------------------------

_main_out_files = w.Box()

#======================================================

# Top buttons
_start_files = w.Button(icon = 'expand', tooltip = 'refresh interface', layout=w.Layout(width = '35px', height = '30px'))
_close_files = w.Button(icon = 'times', tooltip = 'close interface', layout=w.Layout(width='35px', height = '30px'))

# Children of main interface
_filestrument_out = w.Output(layout = w.Layout(
                                            height = '670px', width = '99%', 
                                            align_items='center', justify_content='flex-start'))
_load_as_out_main = w.Output(layout = w.Layout(
                                            height = '670px', align_items='center', 
                                            justify_content='flex-start',
                                            width = '99%', overflow = 'clip'))

#======================================================

# Determine the operating system
_os = platform.system()

#======================================================

# Set code colors and highlights

_grey_font = "<font color = grey>"
_green_font = "<font color = #17a6a6>"
_green_font2 = "<font color = #00b100>"
_red_font = "<font color = #fb657e>"
_blue_font = "<font color = #17b8ee>"
_violet_font = "<font color = #be60fd>"


_bisque_highlight = '<mark style="background-color: bisque">' + _grey_font
_green_highlight = '<mark style="background-color: #c3eaea">' + _grey_font
_skyblue_highlight = '<mark style="background-color: #c8edf9">' + _grey_font
_grey_highlight = '<mark style="background-color: #e0e0e0">' + _grey_font
_red_highlight = '<mark style="background-color: #eeb4b6">' + _grey_font
_violet_highlight = '<mark style="background-color: #f4c3f1">' + _grey_font

_no_highlight = '</mark>'

#======================================================

_filestrument_out_font = '<div style="font-family: Comic Sans MS; font-size: 14px; line-height: 2.4"><b>'
_load_as_font = '<div style="font-family: Comic Sans MS; font-size: 16px; line-height: 1.5"><b>'

_element_sep = w.HTML(value="<h4><b><font color='grey'>｜")

_stripes_load_files = w.VBox([
            w.Output(layout=w.Layout(border = '4px solid #66d6d4', width = '300px')),
            w.Output(layout=w.Layout(border = '4px solid grey', width = '310px')),
            w.Output(layout=w.Layout(border = '4px solid #f7ce9c', width = '320px'))],
            layout=w.Layout(align_items='flex-start'))

#======================================================

_filestrument_html_out = w.Output(layout=w.Layout(width = '205px', height = '45px',
                                                justify_content = 'center'))

_filestrument_text = '𝕱𝖎𝖑𝖊𝕾𝖙𝖗𝖚𝖒𝖊𝖓𝖙'

_filestrument_html = w.HTML('<b><h2><font color=#25c1b9>' + _filestrument_text[:4] + _grey_font + \
                            _filestrument_text[4:])

with _filestrument_html_out:
    display(_filestrument_html)

#======================================================

_empty_folder_icon = w.Button(icon='folder', disabled=True,
                            style=dict(
                                        font_family="Arial",
                                        font_size="180px",
                                        text_color="#ffa001",
                                        button_color = '#fbf2e5'),
                            layout = dict(
                                        height = '160px', width = 'auto'))

#======================================================

_background_style = f"""
<style>
.background_style {{
    background-color:#fbf2e5;
    border-radius: 60px; 
    border: 4px solid {'darkslategrey'};
    box-shadow: 6px 6px 4px rgba(0, 0, 0, 0.8); /* Shadow effect */
}}
</style>
"""

_filestrument_out.add_class("background_style")
_load_as_out_main.add_class("background_style")


with _filestrument_out:
    display(HTML(_background_style))

with _load_as_out_main:
    display(HTML(_background_style))

#======================================================


def run():
    display(w.VBox([

                w.HBox([
                    _filestrument_html_out, 

                    w.VBox([w.Label(layout=w.Layout(height = '8px')), 

                    w.HBox([_element_sep, _stripes_load_files, _element_sep, 
                            _start_files, _close_files],
                              layout=w.Layout(align_items='center',))]),
                    w.Label(layout=w.Layout(width = '155px'))
                                        ], 
                            layout = w.Layout(justify_content='flex-end', align_items='center')),

                _main_out_files]))
            
#============================================================================================================

# Outputs

_curdir_out = w.Output(layout = w.Layout(height='72px', width = '955px', align_items = "center", justify_content = 'center', overflow='auto'))
_drives_out = w.Output(layout = w.Layout(height = '42px', width = '0px',  justify_content = 'center', display = 'flex', flex_flow = 'col wrap'))
_folders_out = w.Output(layout = w.Layout(height = '320px', justify_content='flex-start'))
_files_out = w.Output(layout = w.Layout(height = '180px', justify_content='flex-end'))
_folder_buttons_out = w.Output(layout = w.Layout(height = '214px', width= '80%', display='flex', align_items = 'center', justify_content='center'))
_search_folder_out = w.Output(layout = w.Layout(height = '35px', align_items = "center", justify_content = 'center'))
_search_files_out = w.Output(layout = w.Layout(height = '35px', align_items = "center", justify_content = 'center'))

#======================================================

# Buttons

_back_one_dir = w.Button(description = 'back', tooltip='back one directory', icon = 'arrow-left', layout = w.Layout(width = 'auto'))
_open_directory = w.Button(description='open', tooltip = 'open directory', icon='external-link', layout=w.Layout(width='auto'))
_expand_files = w.Button(description='expand', layout = w.Layout(width = '80px'))
_load_file = w.Button(description='Load file', icon="download", layout=w.Layout(width='145px', height = '40px'), style=dict(font_size = '16px'))
_open_in_excel = w.Button(description='Open in Excel', icon="file-excel-o", layout=w.Layout(width='145px', height = '40px'), style=dict(font_size = '16px'))
_cancel = w.Button(description = 'Main menu', icon = 'arrow-left', layout = w.Layout(width = '145px', height = '40px'), style=dict(font_size = '16px'))
_copy_code_load_files = w.Button(description='Copy code', icon='clipboard', layout=w.Layout(width='145px', height = '40px'), style=dict(font_size = '16px'))
_short_code = w.Button(description = 'Short',
                     style=w.ButtonStyle(
                             button_color = '#b6e7f7',
                             font_size = '14px',
                             font_weight = 'bold'),
                     layout=w.Layout(width = '80px'))
_full_code = w.Button(description = 'Long',
                     style=w.ButtonStyle(
                             button_color = None,
                             font_size = '14px',
                             font_weight = 'bold'),
                    layout=w.Layout(width = '80px'))

#======================================================

# Other widgets

_curdir_widget = w.HTML()
_folders_message_html = w.HTML()
_files_message_html = w.HTML()
_code_html = w.HTML(layout=w.Layout(height = '503.55px', width='960px'))
_n_to_load_html = w.HTML()
_search_folder = w.Text(placeholder = 'Enter folder name', layout = w.Layout(width='auto'))
_search_files = w.Text(placeholder = 'Enter file name', layout = w.Layout(width='auto'))
_file_list = w.SelectMultiple(continuous_update=False, layout = w.Layout(width = '455px', height='58px'))

#============================================================================================================
#============================================================================================================
#============================================================================================================

with _curdir_out:

    display(_curdir_widget)

#======================================================

with _search_folder_out:
    display(w.HBox([_back_one_dir, _element_sep, _open_directory, _element_sep,
                    w.HTML(value="<h4><b><font color='grey'>Search folders"),
                    _search_folder,
                    _drives_out],
                    layout=w.Layout(align_items='center')))

#========================

with _folders_out: # this is so that we can minimize folders and maximize the files/sheets
        
    display(w.VBox([
                w.Label(value='🅕🅞🅛🅓🅔🅡🅢',
                        style=dict(font_size="22px",
                                font_weight="bold",
                                text_color="#4f90c4"),
                        layout=w.Layout(height='23px',
                                        align_items='center',
                                        justify_content='flex-start')),
        
                _folders_message_html,
        
                _search_folder_out,
                
                _folder_buttons_out],
        
                    layout=w.Layout(align_items='center')))

#========================

with _search_files_out:
    display(w.HBox([_expand_files, _element_sep,
                    w.HTML(value="<h4><b>" + _grey_font + "Search files"),
                    _search_files], layout=w.Layout(align_items='center')))

#========================

with _files_out:
        
    display(w.VBox([w.Label(value='🅕🅘🅛🅔🅢',
                            style=dict(font_size="22px",
                                    font_weight="bold",
                                    text_color="lightseagreen"),
                            layout=w.Layout(height='23px',
                                            align_items='center',
                                            justify_content='flex-start')),
                    
                    _files_message_html,
                    
                    _search_files_out,
                                            
                    _file_list,
                    
                    ], layout = w.Layout(align_items='center')))

#======================================================

with _filestrument_out: # all elements of the load FILES menu 

    display(w.VBox([
                    _curdir_out, 
                    _folders_out,
                    _files_out, 
                    w.HBox([_load_file, _element_sep, _open_in_excel], 
                        layout = w.Layout(align_items='center', justify_content='center'))]))

#========================

with _load_as_out_main:
    
    display(w.VBox([
                        w.HBox([w.HTML(_load_as_font + _grey_font + 'Number of files: '), _n_to_load_html, _element_sep, 
                                w.HTML(_load_as_font + _grey_font + 'Load names: '), _short_code, _full_code], 
                               layout=w.Layout(height = '55px', align_items = 'center', justify_content='center')),
                        w.HBox([w.Output(layout=w.Layout(width = '30px')), _code_html],
                              layout=w.Layout(border_bottom = '2px solid grey', border_top = '2px solid grey')),
                        w.HBox([_cancel, _element_sep, _copy_code_load_files], 
                               layout=w.Layout(justify_content='center', align_items='center', 
                                              height = '55px'))
                    ], 
        layout=w.Layout(width = '900px')))
    
#============================================================================================================
#============================================================================================================
#============================================================================================================

def _filestrument_html_func(a):
    
    for num, i in enumerate(_filestrument_text):
        _filestrument_html.value = '<b><h2>' + _grey_font + _filestrument_text[0:num+1]
        time.sleep(0.04)

    _filestrument_html.value = '<b><h2><font color=#25c1b9>' + _filestrument_text[:4] + \
                            _grey_font + _filestrument_text[4:]

_start_files.on_click(_filestrument_html_func)

#======================================================

def _change_descr_start_files(a):

    _main_out_files.layout.height = '680px'

    _start_files.icon = 'refresh'

_start_files.on_click(_change_descr_start_files)

#=================================================

def _close_files_menu(a):

    _main_out_files.layout.height = '0px'
    
    _start_files.icon = 'expand'
    
_close_files.on_click(_close_files_menu)

#======================================================

def _set_files_as_child(a):

    _main_out_files.children = [_filestrument_out]
        
    _start_files.on_click(_set_files_as_child, remove=True)
    
#============================================================================================================
#============================================================================================================
#============================================================================================================

def _expand_files_func(a):
    
    _folders_out.layout.height = '0px'
    _files_out.layout.height = '500px'
    _file_list.layout.height = '385px'
    
    _expand_files.on_click(_expand_files_func, remove=True)
    _expand_files.on_click(_shrink_files_func)
    
    _expand_files.description = 'shrink'

_expand_files.on_click(_expand_files_func)   

#======================================================

def _shrink_files_func(a):
    
    _folders_out.layout.height = '320px'
    _files_out.layout.height = '180px'
    _file_list.layout.height = '58px'
    
    _expand_files.on_click(_shrink_files_func, remove=True)
    _expand_files.on_click(_expand_files_func)
    
    _expand_files.description = 'expand'

#============================================================================================================
#============================================================================================================
#============================================================================================================

def _go_back_one_directory(a):

    os.chdir('..')
    
_back_one_dir.on_click(_go_back_one_directory)

#======================================================

# Update what is shown in the path
def _update_curdir(a):
    
    global _dummy_directory # global because it is used in check_dummy_directory function later
    
    _dummy_directory = os.getcwd().replace('\\', '/').replace("'", r"\'")

    _curdir_widget.value = '<div style="font-family: Comic Sans MS; font-size: 18px;"><b>' + \
                        _grey_font + "'" + \
                        '/'.join(_dummy_directory.split('/')[:-1]) + '/' + \
                        '<font color = saddlebrown>' + _dummy_directory.split('/')[-1] + \
                        _grey_font + "'"

_start_files.on_click(_update_curdir)
_back_one_dir.on_click(_update_curdir)

# _set_files_as_child function has to come AFTER _update_curdir (for some strange reason)
_start_files.on_click(_set_files_as_child)
_cancel.on_click(_set_files_as_child)

#======================================================

def _open_directory_func(a):
    
    directory_path = os.getcwd()
    
    if _os == 'Windows':
        
        subprocess.run(["start", "explorer", directory_path], shell=True)
        
    elif _os == 'Darwin':
        
        subprocess.run(["open", directory_path])
        
    elif _os == 'Linux':
        
        subprocess.run(["xdg-open", directory_path])
    
_open_directory.on_click(_open_directory_func)

#============================================================================================================
#============================================================================================================
#============================================================================================================

def _show_hard_drives(a):
    
    _file_list.value = () # file_list value has to be reset each time. THIS IS IMPORTANT

    if _os == 'Windows':

        import win32api
        drives = win32api.GetLogicalDriveStrings()
        drives = drives.split('\000')[:-1]

        if len(drives) > 1:

            _drives_out.layout = w.Layout(max_width = '300px')

            drive_ls = []
            
            for i in drives:
                
                button = '_Button_' + i.replace(':\\', '')
                drive_ls.append(button)
                
                globals()[button] = w.Button(description = i.replace('\\', ''), icon='fa-hdd-o', style = w.ButtonStyle(button_color= 'linen'))
                string_width = button + ".layout = w.Layout(width = 'auto', height = '26px', max_width = '100px')"
                exec(string_width)
                
                function = 'def ' + button + '_func(kio):\n    try:\n        os.chdir("'  + \
                            i.replace('\\', '').replace("'", r"\'") + '")\n    except:\n        pass'
                exec(function)
                function = button + '.on_click(' + button + '_func' + ')'
                exec(function)
                
                function = button + '.on_click(_update_curdir)'
                exec(function)

                function = button + '.on_click(_show_folders)'
                exec(function)
                
                function = button + '.on_click(_show_files)'
                exec(function)
                
                function = button + '.on_click(_update_search_value)'
                exec(function)

            with _drives_out:

                clear_output(True)
                display(w.HBox([_element_sep, 
                                w.HBox([globals()[drive_ls[i]] for i in range(len(drive_ls))])]))
        else:
            pass

_start_files.on_click(_show_hard_drives)

#============================================================================================================
#============================================================================================================
#============================================================================================================

def _update_search_value(a):
    
    _search_folder.value = ''
    _search_files.value = ''
        
_start_files.on_click(_update_search_value)
_back_one_dir.on_click(_update_search_value)

#===========================================

def _show_folders(a):
            
    _file_list.value = () # file_list value has to be reset each time. THIS IS IMPORTANT
    
    global _folder_ls
    _folder_ls = []
    
    # delete folder buttons of a directory after one navigates away from it - for memory saving purposes
    try:
        if _folders_to_delete:
            for i in _folders_to_delete:
                del globals()[i]
        else:
            pass
    
    except:
        pass
    
    curdir = os.getcwd().replace('\\', '/')
    
    
    try: # Some (hidden) folders cause error
        
        folders = sorted([next(os.walk('.'))[1][i] for i in np.arange(
                0, len(next(os.walk('.'))[1])) if next(os.walk('.'))[1][i][0] != '.'])

        # -----------------------

        if len(folders) > 50:

            _folders_message_html.value = _filestrument_out_font + \
            _grey_font + 'Showing first ' + '<font color = #4f90c4>' +  '50 ' + \
            _grey_font + 'folders out of ' + '<font color = #4f90c4>'  + str(len(folders)) + \
            _grey_font +' folders. Please access the rest via search'

        else:

            _folders_message_html.value = _filestrument_out_font + \
            _grey_font + 'Number of folders: ' + '<font color = #4f90c4>' + str(len(folders))


        # -----------------------

        for i in folders[:50]:

            button = '_Button_' + re.sub('\W+', '_', i).strip('_')
            _folder_ls.append(button)

            globals()[button] = w.Button(description = i, icon='folder-open', 
                                        style = w.ButtonStyle(button_color = 'bisque', 
                                                            font_size = '14px',
                                                            text_color="#454545"))
            string_width = button + ".layout = w.Layout(width = 'auto', max_width = '300px', height = '39px')"
            exec(string_width)

            function = 'def func_' + button + "(a):\n    try:\n        os.chdir('" + curdir.replace("'", r"\'") + "/" + \
                        i.replace("'", r"\'") + "')\n    except:\n        pass"
            exec(function)

            function = button + '.on_click(func_' + button + ')'
            exec(function)

            function = button + '.on_click(_update_curdir)'
            exec(function)

            function = button + '.on_click(_show_folders)'
            exec(function)

            function = button + '.on_click(_show_files)'
            exec(function)
            
            function = button + '.on_click(_update_search_value)'
            exec(function)

        if _folder_ls != []:
            with _folder_buttons_out:
                clear_output(True)
                display(w.HBox([globals()[_folder_ls[i]] for i in range(len(_folder_ls))],
                                layout = w.Layout(display = 'flex', flex_flow = 'row wrap',
                                                justify_content = 'center', height = '214px',
                                                align_items = 'center')))

            _search_folder.disabled = False
            _folders_to_delete = _folder_ls

        else:
            with _folder_buttons_out:
                clear_output(True)
                display(_empty_folder_icon)
            
            _search_folder.disabled = True

                
    except Exception as e:
        
        with _folder_buttons_out:
            clear_output(True)
            display(_empty_folder_icon)

        _search_folder.disabled = True
                
_start_files.on_click(_show_folders)
_back_one_dir.on_click(_show_folders)

#======================================================

def _search_folder_func(a):
        
    _file_list.value = () # file_list value has to be reset each time. THIS IS IMPORTANT
    
    global _folder_ls
    _folder_ls = []
    
    # delete folder buttons of a directory after one navigates away from it - for memory saving purposes
    try:
        if _folders_to_delete:
            for i in _folders_to_delete:
                del globals()[i]
        else:
            pass
    
    except:
        pass

    curdir = os.getcwd().replace('\\', '/')
        
    try: # Some (hidden) folders cause error
        
        folders = sorted([next(os.walk('.'))[1][i] for i in np.arange(
                0, len(next(os.walk('.'))[1])) if next(os.walk('.'))[1][i][0] != '.'])
        
        # subset is the distinguishing feature between this and _show_folders function
        subset = [i for i in folders if _search_folder.value.lower() in i.lower()]
        
        # -----------------------
        
        if len(subset) > 50:
            
            _folders_message_html.value = _filestrument_out_font + \
            _grey_font + 'Showing first ' + '<font color = #4f90c4>' +  '50 ' + \
            _grey_font + 'folders out of ' + '<font color = #4f90c4>'  + str(len(subset)) + \
            _grey_font +' folders. Please access the rest via search'
                    
        else:
            
            _folders_message_html.value = _filestrument_out_font + \
            _grey_font + 'Number of folders: ' + '<font color = #4f90c4>' + str(len(subset))
            
        # -----------------------

        for i in subset[:50]:

            button = '_Button_' + re.sub('\W+', '_', i).strip('_')
            _folder_ls.append(button)

            globals()[button] = w.Button(description = i, icon='folder-open', 
                                        style = w.ButtonStyle(button_color = 'bisque',
                                                            font_size = '14px',
                                                            text_color="#454545"))
            string_width = button + ".layout = w.Layout(width = 'auto', max_width = '300px', height = '39px')"
            exec(string_width)

            function = 'def func_' + button + "(a):\n    try:\n        os.chdir('" + curdir.replace("'", r"\'") + "/" + \
                        i.replace("'", r"\'") + "')\n    except:\n        pass"
            exec(function)
            
            function = button + '.on_click(func_' + button + ')'
            exec(function)

            function = button + '.on_click(_update_curdir)'
            exec(function)
            
            function = button + '.on_click(_show_folders)'
            exec(function)
            
            function = button + '.on_click(_update_search_value)'
            exec(function)
            
            function = button + '.on_click(_show_files)'
            exec(function)


        if _folder_ls != []:
            with _folder_buttons_out:
                clear_output(True)
                display(w.HBox([globals()[_folder_ls[i]] for i in range(len(_folder_ls))],
                                layout = w.Layout(display = 'flex', flex_flow = 'row wrap',
                                                justify_content = 'center', height = '214px',
                                                align_items = 'center')))

            _search_folder.disabled = False
            _folders_to_delete = _folder_ls

        else:
            with _folder_buttons_out:
                clear_output(True)
                display(_empty_folder_icon)
            
            _search_folder.disabled = True
                
    except Exception as e:
        
        with _folder_buttons_out:
            clear_output(True)
            display(_empty_folder_icon)

        _search_folder.disabled = True
        
    _search_folder.value = ''

#============================================================================================================

def _show_files(a):
    
    global _test_files
    
    try: # hidden/unaccessible folders cause error
        
        files = sorted([i for i in os.listdir() if i.split('.')[-1] in ['xlsx', 'csv']])
        
        if len(files)>= 1:
            
            if len(files) > 50:
                _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Showing first ' + _green_font +  '50 ' + \
                _grey_font + 'files out of ' + _green_font  + str(len(files)) + \
                _grey_font +' files. Please access the rest via search'
                
            else:
                _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Number of files: ' + _green_font  + \
                    str(len(files))

            _file_list.disabled = False
            _file_list.options = (files[:50])
            _search_files.disabled = False
            
        elif len(files)== 0:
            
            _file_list.options = ['', '                                            NO FILES AVAILABLE'] # originally 19
            _file_list.disabled = True
            _search_files.disabled = True
            _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Number of files: ' + _green_font  + \
                    str(len(files))
            
        _test_files = files[:50]
                        
    except:
        pass
    
    _search_files.value = ''
        
_start_files.on_click(_show_files)
_back_one_dir.on_click(_show_files)
_search_folder.on_submit(_show_files)
_cancel.on_click(_show_files)

#==========================================

def _search_files_func(a):
    
    try: # hidden/unaccessible folders cause error
        
        files = sorted([i for i in os.listdir() if i.split('.')[-1] in ['xlsx', 'csv']])
        
        files = [i for i in files if _search_files.value.lower() in i.lower()]
        
        if len(files)>= 1:
            
            if len(files) > 50:
                _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Showing first ' + _green_font +  '50 ' + \
                _grey_font + 'files out of ' + _green_font  + str(len(files)) + \
                _grey_font +' files. Please access the rest via search'
                
            else:
                _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Number of files: ' + _green_font + str(len(files))
                
            _file_list.options = (files[:50])
            _file_list.disabled = False
            
            _search_files.disabled = False

        elif len(files)== 0:
            
            _file_list.disabled = True
            _file_list.options = ['', '                                            NO FILES AVAILABLE'] # originally 19
            
            _search_files.disabled = True
            
            _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Number of files: ' + _green_font  + \
                    str(len(files))
            
    except:
        pass
    
    _search_files.value = ''

#======================================================

# Various settings depending on the LENGTH of the values and options of the _file_list

def _observe_file_list(a):
    
    if len(_file_list.options) < 5:
                
        _expand_files.disabled = True
        
    else:
        
        _expand_files.disabled = False
        
        
    if len(_file_list.value) == 0:
        
        _load_file.disabled=True
        _open_in_excel.disabled=True
            
    else:
        
        _load_file.disabled=False
        _open_in_excel.disabled=False
        
        
    if len(_file_list.value) > 1:
        
        _load_file.description = 'Load files'
    
    else:
        
        _load_file.description = 'Load file'
        
_file_list.observe(_observe_file_list)

#============================================================================================================
#============================================================================================================
#============================================================================================================

# Function that runs when load_file button is pressed

def _prepare_files(a):
    
    _file_list_to_load = []
    
    global _code_to_display_short, _code_to_display_full
    _code_to_display_short, _code_to_display_full = [], []
    
    global _code_to_copy
    
    global _code_to_copy_short, _code_to_copy_full
    _code_to_copy_short, _code_to_copy_full = [], []
    
    global _import_html, _import_code

    #-----------------------------
    
    _n_to_load_html.value = _load_as_font + _green_font + str(len(_file_list.value))

    # Prepare necessary elements

    for num, i in enumerate(_file_list.value):

        try: # In case the file is not openable :) 

            _files_message_html.value = _filestrument_out_font + \
               _grey_font + 'Preparing ' + _red_font + \
               i[:50] + _grey_font + '. Please wait ... '

            time.sleep(0.5)

        #-----------------------------

            # Load as
                        
            load_as = re.sub('[\W_]+', '_', ''.join(i.split('.')[:-1]).lower().strip('_'))

        #-----------------------------

            # Sheet name, Separator and Pandas codes
            
            try: # if the file is xlsx file
                
                sheet_names = load_workbook(i, read_only=True).sheetnames
                sheet_names = [i.replace("'", r"\'") for i in sheet_names]
                
                if len(sheet_names)>1:
                
                    sheet_name = _grey_font + ', sheet_name' + _violet_font + ' = ' + _red_font + "'" + sheet_names[0] + "'" + _grey_font + ')'
                    other_sheets = " (sheets: '" + "', '".join(sheet_names) + "'" + ')</i><br>'

                    
                    sheet_name_to_copy = ', sheet_name = ' + "'" + sheet_names[0] + "'" + ')'
                    other_sheets_to_copy = " (sheets: '" + "', '".join(sheet_names) + "'" + ')\n'
                                         
                    
                else:
                    
                    sheet_name = _grey_font + ')'
                    other_sheets = '<br>'
                    
                    sheet_name_to_copy = ')'
                    other_sheets_to_copy = '\n'
                
                
                separator = ''
                separator_to_copy = ''
                
                pandas_code = 'pd.read_excel('
                    

            except: # if it is a csv file
                
                sheet_name = ''
                other_sheets = '<br>'
                
                sheet_name_to_copy = ''
                other_sheets_to_copy = '\n'

                _temp_data = pd.read_csv(i, error_bad_lines = False,  warn_bad_lines=False, 
                                   skip_blank_lines=True, nrows = 100) # first read the file without a delimiter

                if len(_temp_data.columns) > 1: # this means that the default delmiter works

                    _delimiter_for_csv = ','

                else:

                    if int(pd.Series(_temp_data.columns.str.replace(' ', '')).str.contains(';')):

                        _delimiter_for_csv = ';'

                    elif int(pd.Series(_temp_data.columns.str.replace(' ', '')).str.contains(',')):

                        _delimiter_for_csv = ','

                    elif len(_temp_data.columns)==1 & int(pd.Series(_temp_data.columns.str.replace(
                                                                            ' ', '')).str.contains('\t')):
                        _delimiter_for_csv = '\t'

                del _temp_data
                
                if _delimiter_for_csv!=',':
                    
                    separator = _grey_font + ', sep' + _violet_font + ' = ' + _red_font + "'" + _delimiter_for_csv + "'" + _grey_font + ')'   
                    separator_to_copy = ', sep' + ' = ' + "'" + _delimiter_for_csv + "'" + ')'
                    
                else:
                    separator = _grey_font + ')'
                    separator_to_copy = ')'
                
                pandas_code = 'pd.read_csv('

            #=====================
            
            tail_html = _violet_font + ' = ' + \
                        _grey_font + pandas_code + \
                        _red_font + "'" + i.replace("'", r"\'") + "'" + \
                        sheet_name + \
                        separator + '<br><br>'
            
            
            body_html_short = _green_font + '<i># ' + load_as + \
                              other_sheets + '</i>' +  \
                              _skyblue_highlight +  _grey_font + load_as.split('_')[0] + _no_highlight + \
                              tail_html
            
            body_html_full = _green_font + '<i># ' + load_as + \
                             other_sheets + '</i>' + \
                             _bisque_highlight +  _grey_font + load_as + _no_highlight + \
                             tail_html
            
            
            _code_to_display_short.append(body_html_short)
            _code_to_display_full.append(body_html_full)
            
            
            #=====================
            
            tail_code = ' = ' + pandas_code + "'" + i.replace("'", r"\'") + "'" + \
                        sheet_name_to_copy + \
                        separator_to_copy + '\n\n'
            
            
            body_code_short = '# ' + load_as + other_sheets_to_copy + \
                        load_as.split('_')[0] + \
                        tail_code
            
            body_code_full = '# ' + load_as + other_sheets_to_copy + \
                        load_as + \
                        tail_code
            
            
            _code_to_copy_short.append(body_code_short)
            _code_to_copy_full.append(body_code_full)
            
            
            #=====================
            
            _file_list_to_load.append(i)
            
            #=====================
        
        except KeyboardInterrupt: # what happens when the user interrupts the file load
            
            _file_list.value = ()

            _files_message_html.value = \
                   _filestrument_out_font + \
                   _grey_font + 'Load interrupted ...'

        except:
            
            _files_message_html.value = _filestrument_out_font + \
               _grey_font + 'Unable to process ' + _red_font + \
               i[:50] + _grey_font + '. Please inspect ... '

            break
    
    #=====================
   
    _import_html = _load_as_font + _green_font + '<i><br># ============= Import dataset(s) from ' + os.getcwd().split('/')[-1] +' ============= #</i><br><br>' + \
                  _grey_font + 'os.chdir(' + _red_font + "'" + os.getcwd() + "'" + _grey_font + ')<br><br>'
    
    _import_code = '\n# ============= Import dataset(s) from ' + os.getcwd().split('/')[-1] +  ' ============= #\n\n' + \
                  'os.chdir(' + "'" + os.getcwd() + "'" + ')\n\n'
        
    #=====================

    if len(_file_list.value) == len(_file_list_to_load):

        _code_html.value = _load_as_font + _import_html  + ''.join(_code_to_display_short)
        
        _code_to_copy = [_import_code] + _code_to_copy_short

        _main_out_files.children = [_load_as_out_main]

    else:

        pass

#======================================================

def _display_short_code(a):
    
    _full_code.style.button_color = None
    _short_code.style.button_color = '#b6e7f7'
    
    _code_html.value = _load_as_font + _import_html  + ''.join(_code_to_display_short)
    
    global _code_to_copy
    _code_to_copy = [_import_code] + _code_to_copy_short
    
_short_code.on_click(_display_short_code)


#======================================================

def _display_full_code(a):
    
    
    _full_code.style.button_color = 'bisque'
    _short_code.style.button_color = None
    
    _code_html.value = _load_as_font + _import_html  + ''.join(_code_to_display_full)
    
    global _code_to_copy
    _code_to_copy = [_import_code] + _code_to_copy_full
    
_full_code.on_click(_display_full_code)
    
#====================================================== 

# Function that runs when open_in_excel button is pressed

def _open_in_excel_func(a):
        
    try: # this is so that keyboardinterrupt does not crash the kernel
        
        for i in _file_list.value:
            
            # loop through each selected file and print that it is being opened
            
            _files_message_html.value = _filestrument_out_font + \
                _grey_font + 'Opening ' + _red_font + \
                i[:50] + _grey_font + ' in Excel'
                
            time.sleep(2)
            
            # if it is a Windows OS
            try:
                os.startfile(i)
                
            except: # if it not a Windows OS
                opener = "open" if sys.platform == "darwin" else "xdg-open"
                subprocess.call([opener, i])

                
            files = sorted([i for i in os.listdir() if i.split('.')[-1] in ['xlsx', 'csv']])
        
            if len(files)>= 1:

                if len(files) > 50:
                    _files_message_html.value = _filestrument_out_font + \
                    _grey_font + 'Showing first ' + _green_font +  '50 ' + \
                    _grey_font + 'files out of ' + _green_font  + str(len(files)) + \
                    _grey_font +' files. Please access the rest via search'

                else:
                    _files_message_html.value = _filestrument_out_font + \
                    _grey_font + 'Number of files: ' + _green_font  + \
                        str(len(files))

                    
    except KeyboardInterrupt: # what happens when the user interrupts the file load
        
        _files_message_html.value = _filestrument_out_font + \
            _grey_font + 'Load interrupted ...'

        pass
            
        _file_list.value = ()

#======================================================

'''This function checks if the path in the interface is the same as the current path.
It is possible that a path is displayed on interface but the user has changed it manually
Moreoever, it also possible that the user has deleted a certain file after it is displayed
in the interface. If any of these cases is true, don't open the file(s) but correct the path, 
refresh the folders and files first instead'''

def _check_dummy_directory(a):

    files = (sorted([i for i in os.listdir() if i.split('.')[-1] in ['xlsx', 'csv']]))
    
    # the paths do not coincide or a file has been deleted
    if (_dummy_directory != os.getcwd().replace('\\', '/').replace("'", r"\'")) | (len([
        i for i in _test_files if i not in files[:50] ]) > 0) | (len([
        i for i in files[:50] if i not in _test_files]) > 0): 

        _load_file.on_click(_update_curdir)
        _load_file.on_click(_show_hard_drives)
        _load_file.on_click(_show_folders)
        _load_file.on_click(_show_files)

        _open_in_excel.on_click(_update_curdir)
        _open_in_excel.on_click(_show_hard_drives)
        _open_in_excel.on_click(_show_folders)
        _open_in_excel.on_click(_show_files)
        
        _search_folder.on_submit(_update_curdir)
        _search_folder.on_submit(_show_hard_drives)
        _search_folder.on_submit(_show_folders)
        _search_folder.on_submit(_show_files)
        
        _search_files.on_submit(_update_curdir)
        _search_files.on_submit(_show_hard_drives)
        _search_files.on_submit(_show_folders)
        _search_files.on_submit(_show_files)
        
        _load_file.on_click(_prepare_files, remove=True)
        _open_in_excel.on_click(_open_in_excel_func, remove=True)
        _search_folder.on_submit(_search_folder_func, remove=True)
        _search_files.on_submit(_search_files_func, remove=True)
        
        _search_folder.value = ''
        _search_files.value = ''

    else:
        
        _load_file.on_click(_update_curdir, remove = True)
        _load_file.on_click(_show_hard_drives, remove = True)
        _load_file.on_click(_show_folders, remove = True)
        _load_file.on_click(_show_files, remove = True)
        
        _open_in_excel.on_click(_update_curdir, remove = True)
        _open_in_excel.on_click(_show_hard_drives, remove = True)
        _open_in_excel.on_click(_show_folders, remove = True)
        _open_in_excel.on_click(_show_files, remove = True)
        
        _search_folder.on_submit(_update_curdir, remove = True)
        _search_folder.on_submit(_show_hard_drives, remove = True)
        _search_folder.on_submit(_show_folders, remove = True)
        _search_folder.on_submit(_show_files, remove = True)
        
        _search_files.on_submit(_update_curdir, remove = True)
        _search_files.on_submit(_show_hard_drives, remove = True)
        _search_files.on_submit(_show_folders, remove = True)
        _search_files.on_submit(_show_files, remove = True)
        
        _load_file.on_click(_prepare_files)
        _open_in_excel.on_click(_open_in_excel_func)
        _search_folder.on_submit(_search_folder_func)        
        _search_files.on_submit(_search_files_func)

_start_files.on_click(_check_dummy_directory)
_load_file.on_click(_check_dummy_directory)
_open_in_excel.on_click(_check_dummy_directory)
_search_folder.on_submit(_check_dummy_directory)
_search_files.on_submit(_check_dummy_directory)

#======================================================

def _back_to_main_menu(a):
    
    _file_list.value = ()
    
    full_code.style.button_color = None
    short_code.style.button_color = '#b6e7f7'
            
_cancel.on_click(_back_to_main_menu)

#=================================================

def _copy_code_load_files_func(a):

    pyperclip.copy(''.join(_code_to_copy)[:-1])
    
    _copy_code_load_files.icon = 'check'
    time.sleep(0.5)
    _copy_code_load_files.icon = 'clipboard'
    
_copy_code_load_files.on_click(_copy_code_load_files_func)